# Enunciado 

Un ciclo Brayton ideal simple utiliza argón como fluido de trabajo. Al principio de la compresión, P1 = 15 psia y T1 = 80 °F; la temperatura máxima del ciclo es 1 200 °F; y la presión en la cámara de combustión es 150 psia. El argón entra al compresor por una abertura de 3 pies2 a una velocidad de 200 pies/s. Determine la tasa a la cual el ciclo genera entro- pía. La temperatura de la fuente es la misma que la temperatura máxima del ciclo, y la temperatura del sumidero es la misma que la temperatura mínima del ciclo.